# DOD Prohibited Substances Database Analysis

This notebook provides tools to load and analyze the DOD prohibited substances database. We'll explore the data structure, perform basic analysis, and create visualizations to understand the substance database better.

## 1. Import Required Libraries

First, let's import all the necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
# Import standard libraries for data manipulation and analysis
import pandas as pd
import json
import sqlite3
import os
import warnings
warnings.filterwarnings('ignore')

# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 2. Load the Substance Database

Let's load the substance database from the available data sources. We'll try multiple approaches to load the data.

In [ ]:
# Load the substance database from JSON file
def load_substance_database():
    """Load the substance database from available sources."""
    
    # Try loading from docs/data.json first
    json_path = "docs/data.json"
    if os.path.exists(json_path):
        print(f"Loading data from {json_path}...")
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        df = pd.DataFrame(data)
        print(f"✓ Successfully loaded {len(df)} substances from JSON file")
        return df
    
    # Try loading from SQLite database if exists
    db_path = "substances.db"
    if os.path.exists(db_path):
        print(f"Loading data from {db_path}...")
        conn = sqlite3.connect(db_path)
        df = pd.read_sql_query("SELECT * FROM substances", conn)
        conn.close()
        print(f"✓ Successfully loaded {len(df)} substances from SQLite database")
        return df
    
    # Try using the SubstanceDatabase class from generate_docs.py
    try:
        from generate_docs import SubstanceDatabase, Settings
        print("Loading data using SubstanceDatabase class...")
        settings = Settings()
        db = SubstanceDatabase(settings.db_file)
        # This will create the database if it doesn't exist
        print(f"✓ Database initialized at {settings.db_file}")
        return None  # Return None to indicate database exists but may be empty
    except ImportError as e:
        print(f"✗ Could not import database classes: {e}")
        return None

# Load the database
df = load_substance_database()
if df is not None:
    print(f"\nDataset shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
else:
    print("No data loaded - database may need to be populated first")

In [ ]:
df["DISPLAY_NAME"] = df["Name"].str.upper()
df

In [ ]:

from unii_client import UniiDataClient
client = UniiDataClient()
client.download_zip()
# client.get_data_info()

In [ ]:
unii_df = client.load_csv_data('UNII_Records_18Aug2025.txt', sep='\t')
to_pubchem_url = lambda s: f"https://precision.fda.gov/ginas/app/ui/substances/{int(s)}" if not pd.isna(s) else None
to_comptox_url = lambda s: f"https://comptox.epa.gov/dashboard/chemical/details/{s}" if not pd.isna(s) else None
def enhance_unii_data(data_df):
    data_df["UNII_URL"] = data_df.UNII.apply(lambda x: f"https://precision.fda.gov/uniisearch/srs/unii/{x}")
    data_df["COMMONCHEMISTRY_URL"] = data_df.RN.apply(lambda x: f"https://commonchemistry.cas.org/detail?cas_rn={x}")
    data_df["NCATS_URL"] = data_df.UNII.apply(lambda x: f"https://drugs.ncats.io/substance/{x}")
    data_df["GSRS_FULL_RECORD_URL"] = data_df.UNII.apply(lambda x: f"https://precision.fda.gov/ginas/app/ui/substances/{x}")
    data_df["PUBCHEM_URL"] = data_df.PUBCHEM.apply(to_pubchem_url)
    data_df["EPA_COMPTOX_URL"] = data_df.EPA_CompTox.apply(to_comptox_url)
    return data_df

unii_df_enhanced = enhance_unii_data(unii_df)

# unii_df_enhanced[unii_df_enhanced.UNII == "001O2254AC"]
# https://commonchemistry.cas.org/detail?cas_rn=77-20-3
# https://drugs.ncats.io/substance/001O2254AC
# https://precision.fda.gov/ginas/app/ui/substances/001O2254AC
# "https://pubchem.ncbi.nlm.nih.gov/compound/120738"
# https://comptox.epa.gov/dashboard/chemical/details/DTXSID4022575

In [ ]:
pd.DataFrame(unii_df.merge(df, left_on='DISPLAY_NAME', right_on='DISPLAY_NAME').iloc[0])

In [ ]:
def clean_substance_data(df):
    """Clean and preprocess the substance data."""
    if df is None or df.empty:
        print("No data to clean")
        return None
    
    print("=== DATA CLEANING ===")
    original_shape = df.shape
    
    # Create a copy to work with
    df_clean = df.copy()
    
    # Standardize column names (find main name column)
    name_column = None
    for col in ['Name', 'name', 'substance_name']:
        if col in df_clean.columns:
            name_column = col
            break
    
    if name_column and name_column != 'name':
        df_clean['name'] = df_clean[name_column]
        print(f"✓ Standardized name column from '{name_column}' to 'name'")
    
    # Parse JSON fields
    json_fields = ['other_names', 'classifications', 'Reasons', 'References']
    for field in json_fields:
        if field in df_clean.columns:
            try:
                # Parse JSON strings into actual lists/dicts
                df_clean[field + '_parsed'] = df_clean[field].apply(
                    lambda x: json.loads(x) if isinstance(x, str) and x.strip().startswith(('[', '{')) else x
                )
                print(f"✓ Parsed JSON field: {field}")
            except Exception as e:
                print(f"⚠ Could not parse {field}: {e}")
    
    # Clean text fields
    text_fields = ['name', 'Reason', 'reason', 'searchable_name', 'Searchable_name']
    for field in text_fields:
        if field in df_clean.columns:
            # Remove extra whitespace and handle nulls
            df_clean[field] = df_clean[field].astype(str).str.strip()
            df_clean[field] = df_clean[field].replace(['nan', 'None', ''], None)
    
    # Extract simplified reason categories
    reason_col = 'Reason' if 'Reason' in df_clean.columns else 'reason'
    if reason_col in df_clean.columns:
        df_clean['reason_category'] = df_clean[reason_col].apply(extract_reason_category)
        print("✓ Extracted reason categories")
    
    print(f"✓ Cleaning complete. Shape: {original_shape} → {df_clean.shape}")
    return df_clean

def extract_reason_category(reason_text):
    """Extract main category from reason text."""
    if not reason_text or pd.isna(reason_text):
        return 'Unknown'
    
    reason_lower = str(reason_text).lower()
    
    if 'schedule i' in reason_lower or 'schedule 1' in reason_lower:
        return 'Schedule I'
    elif 'schedule ii' in reason_lower or 'schedule 2' in reason_lower:
        return 'Schedule II'
    elif 'schedule iii' in reason_lower or 'schedule 3' in reason_lower:
        return 'Schedule III'
    elif 'wada' in reason_lower:
        return 'WADA Prohibited'
    elif 'dodi' in reason_lower:
        return 'DoD Policy'
    elif 'unapproved' in reason_lower:
        return 'Unapproved Drug'
    else:
        return 'Other'

# Clean the data
if df is not None:
    df_clean = clean_substance_data(df)
else:
    print("No data loaded to clean. Please run the data loading cell first.")

## 5. Basic Data Analysis

Let's perform some basic exploratory data analysis to understand the substance database.

In [ ]:
# Basic analysis of the substance database
if 'df_clean' in globals() and df_clean is not None:
    print("=== BASIC STATISTICS ===")
    print(f"Total substances: {len(df_clean):,}")
    
    # Analyze reason categories
    if 'reason_category' in df_clean.columns:
        print("\n=== PROHIBITION CATEGORIES ===")
        reason_counts = df_clean['reason_category'].value_counts()
        for category, count in reason_counts.items():
            percentage = (count / len(df_clean)) * 100
            print(f"{category:<20}: {count:>5} substances ({percentage:5.1f}%)")
        
        # Create a simple visualization
        plt.figure(figsize=(10, 6))
        reason_counts.plot(kind='bar', color='steelblue', alpha=0.7)
        plt.title('Distribution of Substances by Prohibition Category')
        plt.xlabel('Category')
        plt.ylabel('Number of Substances')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    
    # Analyze missing data patterns
    print("\n=== MISSING DATA ANALYSIS ===")
    missing_data = df_clean.isnull().sum()
    missing_data = missing_data[missing_data > 0].sort_values(ascending=False)
    
    if len(missing_data) > 0:
        print("Fields with missing data:")
        for field, count in missing_data.head(10).items():
            percentage = (count / len(df_clean)) * 100
            print(f"{field:<25}: {count:>5} missing ({percentage:5.1f}%)")
    else:
        print("No missing data found!")
    
    # Sample some interesting substances
    print("\n=== SAMPLE SUBSTANCES ===")
    if 'name' in df_clean.columns:
        sample_substances = df_clean['name'].dropna().sample(min(5, len(df_clean))).tolist()
        for i, substance in enumerate(sample_substances, 1):
            print(f"{i}. {substance}")
    
else:
    print("No cleaned data available. Please run the data cleaning cell first.")

## 6. Filter and Search Substances

Let's create some useful functions to filter and search through the substance database.

In [ ]:
# Use the existing classes to work with substance data
def search_substances_by_name(df, search_term, limit=10):
    """Search for substances by name."""
    if df is None or df.empty:
        return pd.DataFrame()
    
    name_col = 'Name' if 'Name' in df.columns else 'name'
    if name_col not in df.columns:
        print("No name column found")
        return pd.DataFrame()
    
    # Case-insensitive search
    mask = df[name_col].str.contains(search_term, case=False, na=False)
    results = df[mask].head(limit)
    
    print(f"Found {len(results)} substances matching '{search_term}':")
    return results

def filter_by_category(df, category):
    """Filter substances by prohibition category."""
    if df is None or 'reason_category' not in df.columns:
        return pd.DataFrame()
    
    filtered = df[df['reason_category'] == category]
    print(f"Found {len(filtered)} substances in category '{category}'")
    return filtered

def get_substance_details(df, substance_name):
    """Get detailed information about a specific substance."""
    if df is None:
        return None
    
    name_col = 'Name' if 'Name' in df.columns else 'name'
    substance = df[df[name_col].str.contains(substance_name, case=False, na=False)]
    
    if substance.empty:
        print(f"No substance found matching '{substance_name}'")
        return None
    
    if len(substance) > 1:
        print(f"Multiple substances found matching '{substance_name}':")
        print(substance[name_col].tolist())
        return substance
    
    return substance.iloc[0]

# Example usage with the loaded data
if 'df_clean' in globals() and df_clean is not None:
    # Search for testosterone-related substances
    print("=== SEARCH EXAMPLE: Testosterone ===")
    testosterone_results = search_substances_by_name(df_clean, "testosterone", limit=5)
    if not testosterone_results.empty:
        name_col = 'Name' if 'Name' in testosterone_results.columns else 'name'
        for idx, row in testosterone_results.iterrows():
            print(f"- {row[name_col]}")
    
    # Filter by Schedule I substances
    print("\n=== FILTER EXAMPLE: Schedule I ===")
    schedule_i = filter_by_category(df_clean, "Schedule I")
    if not schedule_i.empty:
        print("Sample Schedule I substances:")
        name_col = 'Name' if 'Name' in schedule_i.columns else 'name'
        for substance in schedule_i[name_col].head(3):
            print(f"- {substance}")
else:
    print("No data available for search examples")